In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from lxml import etree
from tqdm.notebook import tqdm
from time import sleep

In [2]:
names = pd.read_csv("authornames-csv.csv")
names

,Number,Name
0,3,"A., Sonay"
1,4,"Aarestrup, Emil"
2,5,"Abarbanell, Stephan"
3,6,"Abbott, David"
4,7,"Äbdülrähmanlı, Näriman"
...,...,...
4956,4959,"Гейне, Генріх"
4957,4960,"Гете, Иоганн Вольфганг"
4958,4961,"Гримм, Братья"
4959,4962,"Захер-Мазох, Леопольд фон"


In [3]:
#Names to list: 
new = names["Name"].to_list()
#shortlist = new[0:999]

In [4]:
#remove fullstop at the end: 
corrected = []

for elem in new: 
    if elem.endswith('.'):
        new = elem.replace('.', '') 
        corrected.append(new) 
    else: 
        corrected.append(elem) 

In [6]:
#Funktion für SRU-Schnittstelle: 
worklist = []
alldata = ""

i=0
base_url = "https://services.dnb.de/sru/authorities" #for GND
    
for i in tqdm(range(0,len(corrected))):
    if i%100 == 0: 
        sleep(3)
    
    searchtext = "PER="+corrected[i]+" and bbg=Tp*"
    parameter = {'version' : '1.1' , 'operation' : 'searchRetrieve' , 'query' : searchtext, 'recordSchema' : "MARC21-xml"}

    
    r1 = requests.get(base_url, params = parameter)
    #print(r.url)

    #Weiter mit Beautiful Soup
    response = BeautifulSoup(r1.content)
    records = response.find_all('record')

    
    query = corrected[i]
    if records:
        numberofrecords = response.find_all('numberofrecords')[0].text
        found = numberofrecords
    else:
        found = "N/A"
        
    result = {"query":query, "found":found} 
    worklist.append(result)
    
    alldata = alldata + r1.text
    

    i+=1

  0%|          | 0/4961 [00:00<?, ?it/s]

In [7]:
#results overview: 
df = pd.DataFrame(worklist)
df

,query,found
0,"A., Sonay",1
1,"Aarestrup, Emil",1
2,"Abarbanell, Stephan",1
3,"Abbott, David",5
4,"Äbdülrähmanlı, Näriman",1
...,...,...
4956,"Гейне, Генріх",N/A
4957,"Гете, Иоганн Вольфганг",N/A
4958,"Гримм, Братья",N/A
4959,"Захер-Мазох, Леопольд фон",N/A


In [8]:
#save result as xml:
with open('alldata.xml', 'w', encoding='utf-8') as f:
    print(alldata, file=f)

In [9]:
#save overview as csv: 
df.to_csv("result_overview.csv")

In [19]:
#find authors with more than one hit: 
multiple = df.loc[(df['found'] != "1") & (df['found'] != "N/A")]
multiple
#multiple.to_csv("multiplehits.csv")

,query,found
3,"Abbott, David",5
12,"Abreu, Carlos",9
17,"Ackermann, Erich",12
18,"Ackermann, Rolf",4
21,"Adam, Gabi",3
...,...,...
4923,"Zimmermann, Katharina",14
4938,"Zöller, Martin",3
4943,"Zuckmayer, Carl",3
4949,"Zweig, Stefan",2


In [20]:
#find authors with no hit: 
empty = df.loc[df['found'] == "N/A"]
empty
#empty.to_csv("nohits.csv")

,query,found
11,"Abraham, Filo M",N/A
13,"Acero, Irene",N/A
24,"Addams, Peter",N/A
25,"Adi Mira, Michaels",N/A
26,"Adib, Mu",N/A
...,...,...
4956,"Гейне, Генріх",N/A
4957,"Гете, Иоганн Вольфганг",N/A
4958,"Гримм, Братья",N/A
4959,"Захер-Мазох, Леопольд фон",N/A
